# 동명이인 찾기
* 대소문자 구분 없음

In [2]:
name = ["Tom", "Jerry", "Mike", "Tom", "Mike", "tom", "tOm"]
# name = ["Tom", "tom"]

In [4]:
lower_name = []
result = []

for x in name:
    lower_name.append(x.lower())

# for x in lower_name:
#     print(x.rfind(lower_name))

In [18]:
set([x.lower() for x in name])

{'jerry', 'mike', 'tom'}

In [2]:
import pickle
with open("./kbo.pkl", "rb") as f:
    player_id_list = pickle.load(f)

In [30]:
print(player_id_list)

['61295', '62925', '50905', '65933', '67991', '63963', '60339', '65462', '67900', '69980', '68928', '68900', '67989', '51905', '51903', '51907', '51904', '63920', '75867', '51901', '67954', '68906', '50992', '78122', '67956', '62947', '69996', '62931', '66965', '69940', '65949', '78813', '65643', '64944', '62907', '74465', '50996', '66928', '69994', '63991', '69974', '67962', '63914', '69962', '68902', '50901', '50923', '76232', '51991', '51996', '76118', '62537', '63905', '68905', '76849', '68910', '51948', '77211', '65964', '62025', '67993', '66968', '60263', '50904', '65937', '63959', '78352', '50900', '78361', '50995', '76720', '61869', '63450', '51902', '73306', '67905', '62966', '68909', '65905', '66963', '69992', '51967', '50911', '51994', '50903', '62655', '63123', '51297', '51269', '66201', '68220', '79290', '68204', '50203', '77263', '50214', '51294', '51230', '67246', '69210', '68200', '65269', '63257', '74206', '78224', '51299', '69104', '51234', '67259', '51257', '79215', 

In [131]:
import pickle
import requests
from bs4 import BeautifulSoup

# 최고 연봉자 구하기 로직 추가
import re
top_salary_player_list = []
top_salary_player = {"선수명" : "-", "연봉" : "0달러"}
# top_salary_player = dict()
def get_top_salary_player(player_info_dict):
    dollar_flag = get_dollar_yn(player_info_dict["연봉"])
    salary = get_salary_int(player_info_dict["연봉"])
    if dollar_flag: salary = exchange_dollar_to_won(salary)
    else: salary = convert_won(salary)
    
    top_dollar_flag = get_dollar_yn(top_salary_player["연봉"])
    top_salary = get_salary_int(top_salary_player["연봉"])
    if top_dollar_flag: top_salary = exchange_dollar_to_won(top_salary)
    else: top_salary = convert_won(top_salary)


    if salary < top_salary: return
    elif salary == top_salary: # 같은 연봉을 가진 선수가 존재할 경우 list에 추가하여 저장
        top_salary_player_list.append({"선수명" : player_info_dict["선수명"], "연봉" : player_info_dict["연봉"]})
    else:
        set_top_salary_player(player_info_dict["선수명"], player_info_dict["연봉"])
        top_salary_player_list.clear()
        top_salary_player_list.append(top_salary_player)
        
def get_dollar_yn(salary):
    return salary.find("달러") != -1

def exchange_dollar_to_won(dollar):
    return dollar * 1100

def convert_won(won):
    return won * 10000

def get_salary_int(str_salary):
    # 정은재 선수의 연봉이 빈 값으로 기재되어 있어 빈 값 처리 추가
    re_salary = re.sub(r'[^0-9]', '', str_salary)
    return int(re_salary) if re_salary.isdecimal() else 0

def set_top_salary_player(player_name, salary):
    top_salary_player["선수명"] = player_name
    top_salary_player["연봉"] = salary
    
def add_dollar_won_alt(salary):
    return "(한화 약 " + str(int(exchange_dollar_to_won(salary) / 10000)) + "만원)"
#######

    
head = {
    'user-agent': 
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36 Edg/92.0.902.84"
}

total = dict()
with open("./kbo.pkl", "rb") as f:
    player_id_list = pickle.load(f)
# player_info의 id 값 리스트
span_id_list = []
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblName")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblBackNo")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblBirthday")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblPosition")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblHeightWeight")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblCareer")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblPayment")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblSalary")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblDraft")
span_id_list.append("cphContents_cphContents_cphContents_playerProfile_lblJoinInfo")

for p_id in player_id_list:
    target_url = "https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=" + p_id
    r = requests.get(target_url, headers=head)
    bs = BeautifulSoup(r.text)
    bs_rt = bs.find("div", class_ = "player_info")
    
    temp_dict = dict()
    span_id_list_count = 0

    for tag in bs_rt.findAll("strong"):
        temp_dict[tag.text[0:len(tag.text)-2]] = bs_rt.find("span", id = span_id_list[span_id_list_count]).text
        span_id_list_count += 1
        
    # 최고 연봉자 구하기 로직 추가
#     print(temp_dict["선수명"] + " : " + temp_dict["연봉"])
    get_top_salary_player(temp_dict)
#     get_top_salary_player({"선수명":"test", "연봉":"65000달러"}) # dollar test case
#     get_top_salary_player({"선수명":"test1", "연봉":"65000달러"}) # 동일 연봉자 test case
#     get_top_salary_player({"선수명":"test2", "연봉":"8000만원"}) # 원화 test case
    
    total[p_id] = temp_dict

# 최고 연봉자 구하기 로직 추가
print("현재 KBO 최고 연봉자는")
for x in top_salary_player_list:
    salary = x["연봉"] + add_dollar_won_alt(get_salary_int(x["연봉"])) if get_dollar_yn(x["연봉"]) else x["연봉"]
    print(f'{x["선수명"]}, {salary}')

현재 KBO 최고 연봉자는
추신수, 270000만원


In [124]:
top_salary_player_list

[]

In [12]:
import operator
temp_total = sorted(total.items(), key = operator.itemgetter(8))

IndexError: tuple index out of range

In [9]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, /, *, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



In [5]:
temp_total

[('50007',
  {'선수명': '문상준',
   '등번호': '107',
   '생년월일': '2001년 03월 14일',
   '포지션': '내야수(우투우타)',
   '신장/체중': '183cm/80kg',
   '경력': '가동초-휘문중-휘문고',
   '입단 계약금': '4000만원',
   '연봉': '3000만원',
   '지명순위': '20 KT 2차 8라운드 72순위',
   '입단년도': ''}),
 ('50010',
  {'선수명': '고영찬',
   '등번호': '110',
   '생년월일': '1999년 08월 14일',
   '포지션': '투수(우투우타)',
   '신장/체중': '183cm/78kg',
   '경력': '상인천초-동인천중-인천고-강릉영동대',
   '입단 계약금': '',
   '연봉': '2700만원',
   '지명순위': '20 KT 육성선수',
   '입단년도': ''}),
 ('50011',
  {'선수명': '최태성',
   '등번호': '111',
   '생년월일': '1997년 02월 12일',
   '포지션': '외야수(우투좌타)',
   '신장/체중': '182cm/82kg',
   '경력': '희망대초-모가중-야탑고-홍익대',
   '입단 계약금': '',
   '연봉': '2700만원',
   '지명순위': '20 KT 육성선수',
   '입단년도': ''}),
 ('50014',
  {'선수명': '윤종휘',
   '등번호': '114',
   '생년월일': '1996년 10월 29일',
   '포지션': '내야수(우투우타)',
   '신장/체중': '168cm/70kg',
   '경력': '동막초-재능중-장안고-홍익대',
   '입단 계약금': '',
   '연봉': '2700만원',
   '지명순위': '20 KT 육성선수',
   '입단년도': ''}),
 ('50030',
  {'선수명': '소형준',
   '등번호': '30',
   '생년월일': '2001년 09월 16일',
  